- Remove non alphanumeric characters for simple training

In [1]:
!pip install --upgrade torch -q 

In [2]:
from transformer import Transformer # this is the local file transformer.py
import torch
import numpy as np
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

bangla_vocabulary = [
    START_TOKEN, ' ', '!', '"', '#', '€', '$', '%', '&', "'", '(', ')', '*', '+', 
    ',', ';', '-', '.', '/', ':', '<', '=', '>', '?', '@',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯'
    'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ',
    'ক', 'খ', 'গ', 'ঘ', 'ঙ',
    'চ', 'ছ', 'জ', 'ঝ', 'ঞ',
    'ট', 'ঠ', 'ড', 'ঢ', 'ণ',
    'ত', 'থ', 'দ', 'ধ', 'ন', 
    'প', 'ফ', 'ব', 'ভ', 'ম',
    'য', 'র', 'ল', 'শ', 'ষ', 
    'স', 'হ', 'ড়', 'ঢ', 'য়', 'ঃ', 'ং', '্',
    'ো', 'ি', 'ৌ', 'ৗ', 'ৎ', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', '।', 
    'ঁ', 'া', '্র', '্ন',
    PADDING_TOKEN, END_TOKEN
]
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [4]:
index_to_bangla = {k:v for k,v in enumerate(bangla_vocabulary)}
bangla_to_index = {v:k for k,v in enumerate(bangla_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [5]:
bn_txt_save_path = '/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/bn_data.txt'
en_txt_save_path = '/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/en_data.txt'

In [6]:
with open(en_txt_save_path, 'r') as file:
    english_sentences = file.readlines()
with open(bn_txt_save_path, 'r') as file:
    bangla_sentences = file.readlines()

# # Limit Number of sentences
# TOTAL_SENTENCES = 200000
# english_sentences = english_sentences[:TOTAL_SENTENCES]
# bangla_sentences = bangla_sentences[:TOTAL_SENTENCES]

english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
bangla_sentences = [sentence.rstrip('\n') for sentence in bangla_sentences]

In [7]:
english_sentences[:3]

["janet’s ducks lay 16 eggs per day. she eats three for breakfast every morning and bakes muffins for her friends every day with four. she sells the remainder at the farmers' market daily for $2 per fresh duck egg. how much in dollars does she make every day at the farmers' market?",
 'a robe takes 2 bolts of blue fiber and half that much white fiber.  how many bolts in total does it take?',
 'josh decides to try flipping a house.  he buys a house for $80,000 and then puts in $50,000 in repairs.  this increased the value of the house by 150%.  how much profit did he make?']

In [8]:
bangla_sentences[:3]

['জেনেটের হাঁসগুলি প্রতিদিন 16টি করে ডিম পাড়ে। তিনি প্রতিদিন প্রাতরাশে তিনটি করে ডিম খান এবং বন্ধুদের জন্য প্রতিদিন চারটি ডিম দিয়ে মাফিন তৈরি করেন। অবশিষ্ট হাঁসের ডিমগুলি তিনি প্রতিদিন কৃষকদের বাজারে প্রতি ডিম $2 দরে বিক্রয় করেন। তিনি কৃষকদের বাজারে প্রতিদিন কত ডলার উপার্জন করেন?',
 'একটি পোশাক তৈরি করতে 2 রিল নীল সুতো এবং তার অর্ধেক সাদা সুতো লাগে। এটি তৈরি করতে মোট কতগুলি রিল লাগবে?',
 'জোশ একটি বাড়ি ফ্লিপ করার সিদ্ধান্ত নিয়েছেন। তিনি $80,000-এ একটি বাড়ি ক্রয় করলেন এবং $50,000 দিয়ে মেরামত করলেন। এর ফলে বাড়িটির মূল্য 150% বৃদ্ধি পেল। তিনি কত ডলার লাভ করলেন?']

In [9]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length bangla: {np.percentile([len(x) for x in bangla_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length bangla: 477.06
97th percentile length English: 468.18


In [10]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(bangla_sentences)):
    bangla_sentence, english_sentence = bangla_sentences[index], english_sentences[index]
    if is_valid_length(bangla_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(bangla_sentence, bangla_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(bangla_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 250
Number of valid sentences: 4


In [11]:
bangla_sentences = [bangla_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [12]:
bangla_sentences[:3]

['গ্রেচেনের কাছে 110টি মুদ্রা আছে। রৌপ্য মুদ্রার থেকে 30টি বেশি স্বর্ণ মুদ্রা আছে। গ্রেচেনের কাছে কতগুলি স্বর্ণ মুদ্রা আছে?',
 'পাঞ্চো একদিনে 20 মাইল হাঁটেন। সপ্তাহান্তে ব্যতিরেকে তিনি 10 মাইল হাঁটেন। এক সপ্তাহে তিনি কত মাইল হাঁটেন?',
 'জন 3 ঘণ্টা ধরে পিচ সংগ্রহ করেন। তিনি এক মিনিটে 2টি পিচ সংগ্রহ করতে পারেন। তিনি কতগুলি পিচ সংগ্রহ করেন?']

In [13]:
import torch

d_model = 512
batch_size = 32
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
kn_vocab_size = len(bangla_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          bangla_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [14]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [15]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, bangla_sentences):
        self.english_sentences = english_sentences
        self.bangla_sentences = bangla_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.bangla_sentences[idx]

In [16]:
dataset = TextDataset(english_sentences, bangla_sentences)

In [17]:
len(dataset)

4

In [18]:
dataset[1]

('pancho walks 20 miles a day. except on weekends when he walks 10 miles. how many miles does he walk in a week?',
 'পাঞ্চো একদিনে 20 মাইল হাঁটেন। সপ্তাহান্তে ব্যতিরেকে তিনি 10 মাইল হাঁটেন। এক সপ্তাহে তিনি কত মাইল হাঁটেন?')

In [19]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [20]:
from rich import print
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[
    (
        'gretchen has 110 coins. there are 30 more gold coins than silver coins. how many gold coins does gretchen 
have?',
        'pancho walks 20 miles a day. except on weekends when he walks 10 miles. how many miles does he walk in a 
week?',
        'john collects peaches for 3 hours.  he can collect 2 peaches a minute.  how many peaches does he 
collect?',
        'james has 6 more candies than robert. john has twice as many candies as robert. if john has 54 candies, 
how many more candies does john have than james?'
    ),
    (
        'গ্রেচেনের কাছে 110টি মুদ্রা আছে। রৌপ্য মুদ্রার থেকে 30টি বেশি স্বর্ণ মুদ্রা আছে। গ্রেচেনের কাছে কতগুলি স্বর্ণ মুদ্রা আছে?',
        'পাঞ্চো একদিনে 20 মাইল হাঁটেন। সপ্তাহান্তে ব্যতিরেকে তিনি 10 মাইল হাঁটেন। এক সপ্তাহে তিনি কত মাইল হাঁটেন?',
        'জন 3 ঘণ্টা ধরে পিচ সংগ্রহ করেন। তিনি এক মিনিটে 2টি পিচ সংগ্রহ করতে পারেন। তিনি কতগুলি পিচ সংগ্রহ করেন?',
        'জেমসের কাছে রবার্টের থেকে 6টি ক্যান্ডি বেশি আছে। জনের কাছে রবার্টের থেকে দুইগুণ বেশি ক্যান্ডি আছে। জনের কাছে যদি 54টি ক্যান্ডি থাকে তাহলে জনের কাছে জেমসের থেকে
কতগুলি ক্যান্ডি বেশি আছে?'
    )
]

In [21]:
from torch import nn
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

criterian = nn.CrossEntropyLoss(ignore_index=bangla_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

Modify mask such that the padding tokens cannot look ahead.
In Encoder, tokens before it should be -1e9 while tokens after it should be -inf.
 

Note the target mask starts with 2 rows of non masked items: https://github.com/SamLynnEvans/Transformer/blob/master/Beam.py#L55


In [24]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                     kn_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == bangla_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"bangla Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
              if idx == bangla_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_bangla[idx.item()]
            print(f"bangla Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_bangla[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Inference

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_bangla[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]

In [ ]:
translation = translate("what should we do when the day starts?")
print(translation)
#ದಿನ ಪ್ರಾರಂಭವಾದಾಗ ನಾವು ಏನು ಮಾಡಬೇಕು?

ಇದರ ಬಗ್ಗೆ ಏನು ಮಾಡಬೇಕು?<END>


In [ ]:
translation = translate("how is this the truth?")
print(translation)
#ಇದು ಹೇಗೆ ಸತ್ಯ

ಹೇಗೆ ಇದು ಹೇಗೆ ಹೇಗೆ?<END>


In [ ]:
translation = translate("the world is a large place with different people")
print(translation)
#ಪ್ರಪಂಚವು ವಿಭಿನ್ನ ಜನರೊಂದಿಗೆ ದೊಡ್ಡ ಸ್ಥಳವಾಗಿದೆ

ಇದರಿಂದ ಮೂಲಕ ಸಂಬಂಧಿಸಿದ ಮೇಲೆ ಮಾಡಿದ್ದಾರೆ<END>


In [ ]:
translation = translate("my name is ajay")
print(translation)
#ನನ್ನ ಹೆಸರು ಅಜಯ್

ನಾನು ಕುಟುಂಬದ ಹೆಸರು<END>


In [ ]:
translation = translate("i cannot stand this smell")
print(translation)
#ನಾನು ಈ ವಾಸನೆಯನ್ನು ಸಹಿಸುವುದಿಲ್ಲ

ನಾನು ಅಂತರ ಸಂಗತಿ ನಾನು ಕೊಡುವುದಿಲ್ಲ<END>


In [ ]:
translation = translate("noodles are the best")
print(translation)

ಯಾವುದೇ ಕಾರಣಗಳು ಇಲ್ಲ<END>


In [ ]:
translation = translate("why care about this?")
print(translation)

ಏಕೆ ಕಾರಣ ಏನು?<END>


This translated pretty well : "What is the reason. Why" without punctuation.

In [ ]:
translation = translate("this is the best thing ever")
print(translation)
# ಇದು ಎಂದೆಂದಿಗೂ ಉತ್ತಮವಾಗಿದೆ

ಇದು ಅತ್ಯಂತ ಹೊರತಾಗಿದೆ<END>


The translation : "This is very unusual"

In [ ]:
translation = translate("i am here")
print(translation)
# ನಾನು ಇಲ್ಲಿದ್ದೇನೆ

ನಾನು ಕೇಳಿದ್ದೇನೆ.<END>


Translation: "I have heard". 
This is why word based translator may perform better than character translator. This is actually very good at optimizing the objective of the current transformer even though the translation is off.

In [ ]:
translation = translate("click this")
print(translation)
# ಇದನ್ನು ಕ್ಲಿಕ್ ಮಾಡಿ

ಕ್ಯಾನ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಮಾಡಿ<END>


In [ ]:
translation = translate("where is the mall?")
print(translation)

ಎಲ್ಲಿ ಎಲ್ಲಿ ಎಲ್ಲಿ?<END>


In [ ]:
translation = translate("what should we do?")
print(translation)

ಏನು ಮಾಡಬೇಕು?<END>


This is correct; but it absolutely fumbles on the next one

In [ ]:
translation = translate("today, what should we do")
print(translation)

ಅದನ್ನು ಮಾಡಿದ ಮೇಲೆ ಮಾಡಿದ ಮಾಡಿದ್ದಾರೆ<END>


In [ ]:
translation = translate("why did they activate?")
print(translation)
# ಅವರು ಏಕೆ ಸಕ್ರಿಯಗೊಳಿಸಿದರು?

ಅದರ ಮೇಲೆ ಏಕೆ ಮಾಡಿದ್ದಾರೆ?<END>


In [ ]:
translation = translate("why did they do this?")
print(translation)
# ಅವರು ಇದನ್ನು ಏಕೆ ಮಾಡಿದರು?

ಅವರು ಏಕೆ ಅವರು ಏಕೆ ಮಾಡಿದ್ದಾರೆ?<END>


That turned out well!

In [ ]:
translation = translate("i am well.")
print(translation)
# ನಾನು ಆರಾಮವಾಗಿದ್ದೇನೆ

ನಾನು ನಾನು ಕೊಡುತ್ತೇನೆ.<END>


Translation: "I will give you something"

In [ ]:
translation = translate("whats the word on the street?")
print(translation)

ಇದರ ಬಗ್ಗೆ ಏನು?<END>


Kind of close semantically. Translation is something like: "What is this about"

## Insights

- When training, we can treat every alphabet as a single unit instead of splitting it into it's corresponding parts to preserve meaning. For example, ಮಾ should be 1 unit when comuting a loss. It should not be decomposed into ಮ + ఆ
- Using word-based or BPE based tokenizations may help mitigate (1). Also, we will get valid word (or BPE) units if we do so. 
- Make sure the training set has a large variety of sentences that are not just about one topic like "work" and "government"
- Increase the number of encoder / decoder units for better translations. It was set to the minimum of 1 of each unit here.
- Create a translator with a language you understand ideally.

Overall, this model definately learned something. And you can use other languages instead of this kannada language and might see better luck